In [ ]:
# Project #1 Juypter Notebook
# Created by Melinda Eudy, Cameron Farquhar, Markita Francis, Luis Rojas, Kelly Rose
# Project : 2019 Crime Data Used from several resources ---UCR Crime Rates by County, US Census, and 

In [80]:
# Dependencies and Setup *******Review before final presentation and remove what is not needed*********
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
from functools import reduce
import json
import ctypes  # An included library with Python install.
import datetime

# Import API key
#from config import apikey

In [81]:
# Part 1 - open UCR Crime Rates by County.csv file
#----------open 37059-Codebook County Codes.csv file and merge into first file to obtain County Names 
UCR_Crime = "Resources/UCR_Crime_Rates_by_County.csv"

# Read file and store in Pandas data frame
UCR_Crime_df = pd.read_csv(UCR_Crime)
UCR_Crime_df.head()
#UCR_Crime_df.count() ----all columns had same count

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,...,SVIOL,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON
0,9999,1,8,1,1,1,4863300,4816193,54499,3,...,0,0,0,0,0,0,0,0,0,0
1,9999,1,8,2,1,3,4863300,4816193,207584,14,...,0,0,0,0,0,0,0,0,0,0
2,9999,1,8,3,1,5,4863300,4816193,25778,5,...,0,0,0,0,0,0,0,0,0,0
3,9999,1,8,4,1,7,4863300,4816193,22474,4,...,0,0,0,0,0,0,0,0,0,0
4,9999,1,8,5,1,9,4863300,4816193,57565,6,...,0,0,0,0,0,0,0,0,0,0


In [82]:
# FIPS-ST is the first number of the Code, FIPS_City is the last three numbers of the code.
#-----Make FIPS_City three digits wide (adding zeros in front) and combine the two columns
UCR_Crime_df["FIPS_ST"] = UCR_Crime_df["FIPS_ST"].apply(str)
UCR_Crime_df["FIPS_CTY"] = UCR_Crime_df["FIPS_CTY"].apply(str)
UCR_Crime_df["FIPS_CTY"] = UCR_Crime_df["FIPS_CTY"].apply(lambda x: x.zfill(3))
UCR_Crime_df["CODE"] = UCR_Crime_df[["FIPS_ST", "FIPS_CTY"]].agg("".join, axis=1)
UCR_Crime_df.head()

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,...,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON,CODE
0,9999,1,8,1,1,001,4863300,4816193,54499,3,...,0,0,0,0,0,0,0,0,0,1001
1,9999,1,8,2,1,003,4863300,4816193,207584,14,...,0,0,0,0,0,0,0,0,0,1003
2,9999,1,8,3,1,005,4863300,4816193,25778,5,...,0,0,0,0,0,0,0,0,0,1005
3,9999,1,8,4,1,007,4863300,4816193,22474,4,...,0,0,0,0,0,0,0,0,0,1007
4,9999,1,8,5,1,009,4863300,4816193,57565,6,...,0,0,0,0,0,0,0,0,0,1009


In [83]:
# Finalize file for merging
#-----remove columns: STUDYNO, EDITION, PART, FLIPS_ST, FLIPS_CTY
#-----move Code to the front of the file - ensure that it is a int64.
UCR_Crime_df.drop(columns = ["STUDYNO", "EDITION", "PART", "FIPS_ST", "FIPS_CTY"], inplace = True)
UCR_Crime_df["CODE"] = UCR_Crime_df["CODE"].apply("int64")
pd.set_option("precision",0)
collist = UCR_Crime_df.columns.tolist()
collist.insert(0, collist.pop(collist.index("CODE")))
UCR_Crime_df = UCR_Crime_df.reindex(columns = collist)
UCR_Crime_df.head()

,CODE,IDNO,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,SVIOL,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON
0,1001,1,4863300,4816193,54499,3,3,100,0,0,0,0,0,0,0,0,0,0
1,1003,2,4863300,4816193,207584,14,14,100,0,0,0,0,0,0,0,0,0,0
2,1005,3,4863300,4816193,25778,5,4,94,0,0,0,0,0,0,0,0,0,0
3,1007,4,4863300,4816193,22474,4,5,96,0,0,0,0,0,0,0,0,0,0
4,1009,5,4863300,4816193,57565,6,6,98,0,0,0,0,0,0,0,0,0,0


In [84]:
UCR_Codes = "Resources/Codebook_County_Codes.csv"

# Read file and store in Pandas data frame
UCR_Codes_df = pd.read_csv(UCR_Codes)
#UCR_Codes_df.count()   --- all columns had the same count
UCR_Codes_df.head()

,State,Code,County,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Alabama,1001,Autauga,NaN,NaN,NaN
1,Alabama,1003,Baldwin,NaN,NaN,NaN
2,Alabama,1005,Barbour,NaN,NaN,NaN
3,Alabama,1007,Bibb,NaN,NaN,NaN
4,Alabama,1009,Blount,NaN,NaN,NaN


In [85]:
# Before mergining data:
# ------Remove Unnamed:3 - Unnamed:5
# ------Remove decimal in Code
#-------Rename Code to CODE
UCR_Codes_df.drop(columns = ["Unnamed: 3", "Unnamed: 4", "Unnamed: 5"], inplace = True)
UCR_Codes_df.rename(columns={"Code":"CODE", "State":"STATE", "County":"COUNTY"}, inplace = True)
UCR_Codes_df["CODE"] = UCR_Codes_df["CODE"].apply("int64")
pd.set_option("precision",0)
UCR_Codes_df.head()

,STATE,CODE,COUNTY
0,Alabama,1001,Autauga
1,Alabama,1003,Baldwin
2,Alabama,1005,Barbour
3,Alabama,1007,Bibb
4,Alabama,1009,Blount


In [86]:
# Merge Codebook_County_Codes.csv into UCR_Crime_Rates_by_County.csv
merged_UCR_Crime_df = pd.merge(UCR_Codes_df, UCR_Crime_df,on="CODE", how="right")
merged_UCR_Crime_df.head(15)

,STATE,CODE,COUNTY,IDNO,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,SVIOL,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON
0,Alabama,1001,Autauga,1,4863300,4816193,54499,3,3,100,0,0,0,0,0,0,0,0,0,0
1,Alabama,1003,Baldwin,2,4863300,4816193,207584,14,14,100,0,0,0,0,0,0,0,0,0,0
2,Alabama,1005,Barbour,3,4863300,4816193,25778,5,4,94,0,0,0,0,0,0,0,0,0,0
3,Alabama,1007,Bibb,4,4863300,4816193,22474,4,5,96,0,0,0,0,0,0,0,0,0,0
4,Alabama,1009,Blount,5,4863300,4816193,57565,6,6,98,0,0,0,0,0,0,0,0,0,0
5,Alabama,1011,Bullock,6,4863300,4816193,10634,2,2,95,0,0,0,0,0,0,0,0,0,0
6,Alabama,1013,Butler,7,4863300,4816193,19952,4,4,97,0,0,0,0,0,0,0,0,0,0
7,Alabama,1015,Calhoun,8,4863300,4816193,114022,9,10,100,0,0,0,0,0,0,0,0,0,0
8,Alabama,1017,Chambers,9,4863300,4816193,34047,4,4,98,0,0,0,0,0,0,0,0,0,0
9,Alabama,1019,Cherokee,10,4863300,4816193,25774,4,5,99,0,0,0,0,0,0,0,0,0,0


In [87]:
# Bring in data from the US census per county
#------population density per county
#------median income per county